## Variational Quantum Boltzmann Machine - Qiskit Implementation 

### Imports

In [23]:
# General Imports
import numpy as np
from scipy.linalg import expm

# Execution Imports
from qiskit import Aer
from qiskit.aqua import QuantumInstance

# Circuit Imports
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit.circuit import Parameter

# Operator Imports
from qiskit.aqua.operators import I, Z, StateFn, CircuitStateFn, SummedOp, CircuitSampler, Gradient, ListOp, QFI
from qiskit.aqua.operators.gradients import NaturalGradient, DerivativeBase
from qiskit.aqua.operators.gradients.circuit_qfis import LinCombFull
from qiskit.aqua.operators.gradients.circuit_gradients import LinComb

# Additional Imports
from qiskit.quantum_info import state_fidelity, partial_trace, Statevector
from qiskit.aqua.components.optimizers import SPSA, CG, ADAM, COBYLA

## Gibbs state preparation

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

In [5]:
# Temperature
k_BT = 1

# Evolution time
t =  1/(2*k_BT)

# Define the model Hamiltonian 
H = SummedOp([0.3 * Z^Z^ I^I, 0.2 * Z^I^ I^I,  0.5 * I^Z^ I^I]) 

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

In [6]:
# Instantiate the model ansatz
depth = 1
entangler_map = [[i+1, i] for i in range(H.num_qubits - 1)]
ansatz = EfficientSU2(4, reps=depth, entanglement = entangler_map)
qr = ansatz.qregs[0]
for i in range(int(len(qr)/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Initialize the Ansatz parameters
param_values_init = np.zeros(2* H.num_qubits * (depth + 1))
for j in range(2 * H.num_qubits * depth, int(len(param_values_init) - H.num_qubits - 2)):
    param_values_init[int(j)] = np.pi/2.
    

#### Initial State

The Ansatz $|\psi\left(\omega\left(\tau\right)\right)\rangle$ is initialized such that the first two qubits are in a maximally mixed state.

In [7]:
print('Initial parameters ', param_values_init)

# Initial State

print('\n Circuit ', ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))))

print('\n Full statevector ', CircuitStateFn(ansatz.assign_parameters \
                                          (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data)

print('\n Maximally mixed state', partial_trace(CircuitStateFn(ansatz.assign_parameters\
                        (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data, [0, 1]).data)

Initial parameters  [0.         0.         0.         0.         0.         0.
 0.         0.         1.57079633 1.57079633 0.         0.
 0.         0.         0.         0.        ]

 Circuit       ┌─────────┐┌─────────┐┌───┐┌──────────┐┌─────────┐                       »
q_0: ┤ RY(0.0) ├┤ RZ(0.0) ├┤ X ├┤ RY(pi/2) ├┤ RZ(0.0) ├───────────────────────»
     ├─────────┤├─────────┤└─┬─┘└──┬───┬───┘├─────────┴┐┌─────────┐           »
q_1: ┤ RY(0.0) ├┤ RZ(0.0) ├──■─────┤ X ├────┤ RY(pi/2) ├┤ RZ(0.0) ├───────────»
     ├─────────┤├─────────┤        └─┬─┘    └──┬───┬───┘├─────────┤┌─────────┐»
q_2: ┤ RY(0.0) ├┤ RZ(0.0) ├──────────■─────────┤ X ├────┤ RY(0.0) ├┤ RZ(0.0) ├»
     ├─────────┤├─────────┤                    └─┬─┘    ├─────────┤├─────────┤»
q_3: ┤ RY(0.0) ├┤ RZ(0.0) ├──────────────────────■──────┤ RY(0.0) ├┤ RZ(0.0) ├»
     └─────────┘└─────────┘                             └─────────┘└─────────┘»
«               
«q_0: ──■───────
«       │       
«q_1: ──┼────■──
«     ┌─┴─┐  │  


#### Let's define the target observable consisting of the Ansatz and the Hamiltonian

$$ \langle \psi\left(\omega\left(\tau\right)\right)|H|\psi\left(\omega\left(\tau\right)\right)\rangle $$

In [8]:
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz   
# Use statevector simulation
ansatz_op = CircuitStateFn(ansatz)
op = ~StateFn(H) @ ansatz_op

print('\n Energy expectation value of the initial state ', op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))).eval())


 Energy expectation value of the initial state  0j


#### Target state

$\rho^{target} = \frac{e^{H\otimes I}/{k_BT}}{Z}$

In [9]:
# Compute the density matrix corresponding to the target Gibbs state
h_mat = H.to_matrix()
gibbs_target = expm(-h_mat*t) / np.trace(expm(-h_mat*t))
gibbs_target = partial_trace(gibbs_target, [0, 1]).data

print('Target state ', gibbs_target)

Target state  [[0.14517971+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.32310338+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.23936087+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.29235603+0.j]]


### Define the parameter propagation rule according to McLachlan's variational principle

In [12]:
def get_gibbs_state_params(op, ansatz, param_values, time_steps):

    # Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
    nat_grad = NaturalGradient(grad_method = 'lin_comb', regularization = 'ridge').convert(op, ansatz.ordered_parameters)

    # Propagate the Ansatz parameters step by step according to the explicit Euler method
    for step in time_steps:
        param_dict = dict(zip(ansatz.ordered_parameters, param_values))
        nat_grad_result = np.real(nat_grad.assign_parameters(param_dict).eval())
        param_values = list(np.subtract(param_values, t/num_time_steps * np.real(nat_grad_result)))
    return param_values


### Run the parameter propagation

In [13]:
# Define the discretization grid of the time steps
num_time_steps = 10
time_steps = np.linspace(0, t, num_time_steps)
param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)
    
print('Final parameter values', param_values)

Final parameter values [0.3748650466122236, -0.4353524791155647, -0.009065656466451948, -0.006839849634978346, -0.0033220810527533192, -0.002457020025064206, -0.0023036851404666255, -0.0023040331707213993, 1.9648329484865095, 1.7018832179912435, -0.009609253326270464, -0.008500319965978215, -0.011221011487478671, -0.011087023205375973, -0.0023059259135004067, -0.0023045517780853906]


#### Check the fidelity between trained and target state

In [14]:
# Compute the density matrix corresponding to the final Gibbs state    
gibbs_state = Statevector.from_instruction(ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))))
gibbs_state = partial_trace(gibbs_state, [0, 1])
print('Gibbs state ', np.around(gibbs_state.data, 3))

print('Target state ', np.around(gibbs_target.data, 3))

# Evaluate the fidelity between the trained and the target state
fidelity = state_fidelity(np.around(gibbs_target.data, 3), np.around(gibbs_state.data, 3), validate=False)

print('Fidelity between trained and target state ', fidelity)

Gibbs state  [[ 0.018-0.j  0.   -0.j -0.001+0.j -0.001+0.j]
 [ 0.   +0.j  0.499-0.j -0.002-0.j -0.004-0.j]
 [-0.001-0.j -0.002+0.j  0.151-0.j -0.004+0.j]
 [-0.001-0.j -0.004+0.j -0.004-0.j  0.332-0.j]]
Target state  [[0.145+0.j 0.   +0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.323+0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.239+0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.292+0.j]]
Fidelity between trained and target state  0.9098183229613624


## Train a generative QBM 
More explicitly, we train here a fully visible, diagonal, generative QBM using gradient-free optimization.

### Initialize a parameterized Hamiltonian and the target PDF

In [15]:
a = Parameter('a')    
b = Parameter('b')
c = Parameter('c')  

# Define the model Hamiltonian with parameters
H = SummedOp([a * Z^Z^I^I, b * Z^I^ I ^ I, c * I ^ Z^ I ^ I]) 

H_coeffs = [a, b, c]

# Define the target PDF
p_target = [0.5, 0, 0, 0.5] 

#### Choose a backend

In [16]:
backend = Aer.get_backend('qasm_simulator')
qi = QuantumInstance(backend)

In [17]:
# Compute the gradient of the QFI (A) w.r.t. the Hamiltonian coefficients
def get_dH_qfi(qfi, params, dH_params):
    dH_qfi = []
    for i in range(len(H_coeffs)):
        dH_qfi_term = []
        for param_s in params:
                dH_qfi_term += [dH_params[i] * Gradient(method='lin_comb').convert(qfi, param_s)]
        dH_qfi.append(SummedOp(dH_qfi_term))
    return ListOp(dH_qfi)

In [18]:
# Compute the gradient of the energy (C) w.r.t. the Hamiltonian coefficients
def get_dH_grad(grad, params, dH_params):
    dH_grad = []
    for i, coeff_i in enumerate(H_coeffs):
        dH_grad_term = [Gradient().convert(grad, coeff_i)]
        for param_s in params:
                dH_grad_term += [dH_params[i] * Gradient(method='lin_comb').convert(grad, param_s)]
        dH_grad.append(SummedOp(dH_grad_term))
    return ListOp(dH_grad)

In [24]:
# Prepare the Gibbs state and compute the gradient of the Ansatz parameters w.r.t. the 
# Hamiltonian coefficients while doing so

def get_gibbs_autograd_qbm(op, ansatz, coeffs_values, param_values, time_steps):

    # Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
    nat_grad = NaturalGradient(method = 'lin_comb', regularization = 'ridge').convert(op, ansatz.ordered_parameters)
    qfi = QFI().convert(ansatz_op, ansatz.ordered_parameters)
    grad = Gradient().convert(op, ansatz.ordered_parameters)
    
    # Get the time derivative of the the gradient of the Ansatz parameters w.r.t. the 
    # Hamiltonian coefficients
    def get_dH_dt_params(A, dH_A, dH_C, dt_weights):
        """
        Args:
            A: operator described in arXiv:1804.03023
            dH_A: derivative of A w.r.t. the Hamiltonian weights (parameters)
            dH_C: derivative of C w.r.t. the Hamiltonian weights (parameters)

        Returns: derivative of the Ansatz circuit's weights (parameters) w.r.t. the Hamiltonian weights (parameters)

        """
        #QFI - A: kxk
        #gradient of the energz - dH_C: kxj
        #np.tensordot(dH_A, dt_weights): kxj - with dt_weights: kx1, dH_A: jxkxk
        #k - num params in the VarForm used for VarQITE
        #j - numb terms in the Hamiltonian = dim of the H params

        temp = np.tensordot(dH_A, np.reshape(dt_weights, (len(dt_weights), 1)), axes=1)
        temp = np.transpose(temp)
        temp.shape = (np.shape(dH_C))

        # Solve the respective SLE
        if np.linalg.matrix_rank(A) >= 2 and np.linalg.cond(A) < 1000:
            dH_dt_weights = np.linalg.solve(A, dH_C - temp)
        else:
            dH_dt_weights = NaturalGradient._regularized_sle_solver(A, dH_C - temp, regularization='ridge')

        # print('dH dt weights', dH_dt_weights)
        return dH_dt_weights
    
    dH_param_values = np.zeros((len(H.params), len(ansatz.ordered_parameters)))

    # Propagate the Ansatz parameters step by step according to the explicit Euler method
    for step in time_steps:
        params_dict = {}
        for i, coeff in enumerate([a, b, c]):
            params_dict[coeff] = np.real(coeffs_values[i])
        for j, param in enumerate(ansatz.ordered_parameters):
            params_dict[param] = param_values[j]
        # param_dict = dict(zip(ansatz.ordered_parameters, param_values))
        sampler = CircuitSampler(backend=qi).convert(nat_grad, params_dict)
        nat_grad_result = sampler.eval()[0]
        # Or use the inefficient eval method
        # nat_grad_result = nat_grad.assign_parameters(param_dict).eval()
        
        # 
        dH_qfi = get_dH_qfi(qfi, params, dH_param_values).assign_parameters()
        sampler = CircuitSampler(backend=qi).convert(dH_qfi, params_dict)
        dH_qfi_result = sampler.eval()[0]
        dH_grad = get_dH_grad(grad, params, dH_param_values).assign_parameters().eval()
        sampler = CircuitSampler(backend=qi).convert(dH_grad, params_dict)
        dH_grad_result = sampler.eval()[0]
        
        sampler = CircuitSampler(backend=qi).convert(qfi, params_dict)
        qfi_result = sampler.eval()[0]
        
        # Compute the time derivative of the gradient of the Ansatz parameters w.r.t. the Hamiltonian coefficients
        dH_dt_param_values = get_dH_dt_params(qfi_result, dH_qfi_result, dH_grad_result, nat_grad_result)
        
        # Propagate the parameter values for one time step
        param_values = list(np.subtract(param_values, t/num_time_steps * np.real(nat_grad_result)))
        # Propagate the gradient of the parameter values for one time step
        dH_param_values = list(np.subtract(dH_param_values, t/num_time_steps * np.real(dH_dt_param_values)))
    return param_values, dH_param_values

#### Define the loss function and the optimizer

In [28]:
# Define the loss function
def loss(H_coeffs):
    H_op = H.assign_parameters(dict(zip([a, b, c], np.real(H_coeffs))))
    
    #Combine the measurement and ansatz operator
    op = ~StateFn(H_op) @ ansatz_op
    
    #Prepare the Gibbs state
    param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)    
    # Use the circuit sampler to evaluate the pdf
    sampler = CircuitSampler(backend=qi).convert(ansatz_op, dict(zip(ansatz.ordered_parameters, param_values)))
    p_qbm = sampler.eval()[0].primitive
    # Or use the inefficient eval method
    # p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
    p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)
    print('Trained probability ', p_qbm)
    loss_fn = -np.sum(np.multiply(p_target, np.log(p_qbm)))
    print('loss', loss_fn)
    return np.real(loss_fn)

In [29]:
from qiskit.aqua.operators.gradients import Gradient

# Define the gradient of the loss function
def grad_loss(coeffs_values):
    #Combine the measurement and ansatz operator
    op = ~StateFn(H_op) @ ansatz_op
    
    # Get the Ansatz parameters for the Gibbs state and get the gradient of the Ansatz parameters w.r.t. the
    # Hamiltonian coefficients
    param_values, dH_param_values = get_gibbs_autograd_qbm(op, ansatz, coeffs_values, param_values_init, time_steps)
    # Use the circuit sampler to evaluate the pdf
    sampler = CircuitSampler(backend=qi).convert(ansatz_op, dict(zip(ansatz.ordered_parameters, param_values)))
    p_qbm = sampler.eval()[0].primitive
    
    # Or use the inefficient eval method
    # p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
    # Get the sampling probabilities w.r.t. the computation basis states from the Gibbs state
    p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)
    print('Trained probability ', p_qbm)
    p_qbm = [[min(p, 1e-6) for p in p_items] for p_items in p_qbm]
    
    # Get the function for the gradient of the sampling probabilities w.r.t. the Ansatz parameters
    dparam_p_qbm_fn = Gradient(method = 'param_shift').gradient_wrapper(ansatz_op, ansatz.ordered_parameters, 
                                                                 backend = qi)
    # Get the values for the gradient of the sampling probabilities w.r.t. the Ansatz parameters
    dparam_p_qbm = dparam_p_qbm_fn(param_values)
    
    # Get the values for the gradient of the sampling probabilities w.r.t. the Hamiltonian coefficients
    dH_p_qbm = np.tensordot(np.transpose(dH_param_values), dparam_p_qbm, axes=1).tolist()
    # Get the values for the gradient of the loss function w.r.t. the Hamiltonian coefficients
    try:
        dH_loss = -np.tensordot(np.divide(dH_p_qbm, p_qbm), p_data, axes=1) # jx1
    except Exception:
        dH_loss = -np.tensordot(np.divide(dH_p_qbm, p_qbm), np.transpose(p_data), axes=1)
    print('loss gradient ', dH_loss.flatten())
    return dH_loss.flatten()

### Train the QBM

In [31]:
optimizer = CG(maxiter = 50)
result = optimizer.optimize(len(H_coeffs), loss, gradient_function=grad_loss, initial_point=([-2 , .2, .5]))
print('Trained parameters ', result[0])

TypeError: object of type 'numpy.float64' has no len()

In [ ]:
#Construct the Hamiltonian with the final parameterw
H_op = H.assign_parameters(dict(zip(H_coeffs, [-1.29810568, -0.04292338,  0.1362023])))

#Combine the measurement and ansatz operator
op = ~StateFn(H_op) @ ansatz_op

#Prepare the Gibbs state
param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)

# Get the sampling probabilities
p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)

# Evaluate the l1 norm between the trained and the target state
norm = np.linalg.norm(p_target-p_qbm, ord = 1)

print('L1-norm between trained and target distributions ', norm)

In [ ]:
print(p_qbm)

In [ ]:
a = Parameter('a')    
b = Parameter('b')
c = Parameter('c')  

# Define the model Hamiltonian with parameters
H = (a * Z^Z + b * Z^I - c * I ^ Z) ^ I ^ I

# Define the target PDF
p_target = [0.5, 0, 0, 0.5] 

# Use statevector simulation
ansatz_op = CircuitStateFn(qc = ansatz)
p_qbm = ansatz_op.assign_parameter(dict(zip(ansatz.ordered_parameters, param_values))).eval()


loss = -np.sum(np.multiply(p_target, np.log(p_qbm)))

domega_p_qbm = Gradient().convert(op = ansatz_op, params = ansatz.ordered_parameters, method = 'lin_comb')

qfi = QFI().convert(op = ansatz_op, params = ansatz.ordered_parameters)
from  qiskit.aqua.operator.gradients.gradient.natural_gradient import regularized_sle_solver

dH_omega = regularized_sle_solver(qfi * 0.25, dH_C - dH_A*nat_grad_result, regularization = 'ridge')

dH_p_qbm = domega_p_qbm * dH_omega

dH_loss = -np.tensordot(np.divide(dH_p_qbm, p_qbm), [p_target], axes=1)

In [ ]:
# def get_dH_qfi(qfi, params, dH_params):
#     def get_state_exp(state, param_s, exp=1):
#         state_qc = state.primitive
#         for qreg in state_qc.qregs:
#             if qreg.name == 'work_qubit':
#                 qr_work = qreg
#                 work_q = qreg[0]
#         param_s_gates = state_qc._parameter_table[param_s]
#         for m, param_occurence in enumerate(param_gates):
#             coeffs_s, gates_s = LinComb._gate_gradient_dict(param_occurence[0])[
#                 param_occurence[1]]
#             if exp == 2:
#                 coeffs_s = np.conj(coeffs_s)
#             for k, gate_to_insert_s in enumerate(gates_s):
#                 grad_state = QuantumCircuit(*state_qc.qregs, qr_work)
#                 grad_state.data = state_qc.data
# 
#                 # apply Hadamard on work_q
#                 LinComb.insert_gate(grad_state, param_occurence[0], HGate(),
#                                     qubits=[work_q])
#                 # Fix work_q phase
#                 coeff_s = coeffs_s[k]
#                 sign = np.sign(coeff_s)
#                 is_complex = np.iscomplex(coeff_s)
#                 if sign == -1:
#                     if is_complex:
#                         LinComb.insert_gate(grad_state,
#                                             param_occurence[0],
#                                             SdgGate(),
#                                             qubits=[work_q])
#                     else:
#                         LinComb.insert_gate(grad_state,
#                                             param_occurence[0],
#                                             ZGate(),
#                                             qubits=[work_q])
#                 else:
#                     if is_complex:
#                         LinComb.insert_gate(grad_state,
#                                             param_occurence[0],
#                                             SGate(),
#                                             qubits=[work_q])
# 
#                 # Insert controlled, intercepting gate - controlled by |0>
#                 if exp == 2:
#                     LinComb.insert_gate(grad_state, param_occurence[0],
#                                     XGate(), qubits=[work_qubit], after=False)
# 
#                 if isinstance(param_occurence[0], UGate):
#                     if param_occurence[1] == 0:
#                         LinComb.insert_gate(grad_state, param_occurence[0],
#                                             RZGate(param_occurence[0].params[2]))
#                         LinComb.insert_gate(grad_state, param_occurence[0],
#                                             RXGate(np.pi / 2))
#                         LinComb.insert_gate(grad_state, param_occurence[0],
#                                             gate_to_insert_i,
#                                             additional_qubits=additional_qubits)
#                         LinComb.insert_gate(grad_state, param_occurence[0],
#                                             RXGate(-np.pi / 2))
#                         LinComb.insert_gate(grad_state, param_occurence[0],
#                                             RZGate(-param_occurence[0].params[2]))
# 
#                     elif param_occurence[1] == 1:
#                         LinComb.insert_gate(grad_state, param_occurence[0],
#                                             gate_to_insert_i, after=True,
#                                             additional_qubits=additional_qubits)
#                     else:
#                         LinComb.insert_gate(grad_state, param_occurence[0],
#                                             gate_to_insert_i,
#                                             additional_qubits=additional_qubits)
#                 else:
#                     LinComb.insert_gate(grad_state, param_occurence[0],
#                                         gate_to_insert_i,
#                                         additional_qubits=additional_qubits)
#                 if exp == 2:
#                     LinComb.insert_gate(grad_state, param_occurence[0],
#                                     XGate(), qubits=[work_qubit], after=True)
# 
#                 grad_state = LinCombFull.trim_circuit(grad_state, param_occurence[0])
# 
#                 grad_state.h(work_q)
# 
#                 state = np.sqrt(np.abs(coeff_s)) * state.coeff * CircuitStateFn(grad_state)
# 
#                 # Chain Rule parameter expressions
#                 gate_param = param_occurence[0].params[param_occurence[1]]
#                 if gate_param == param:
#                     state = phase_fix_observable @ state
#                 else:
#                     if isinstance(gate_param, ParameterExpression):
#                         expr_grad = DerivativeBase.parameter_expression_grad(gate_param, param)
#                         state = (expr_grad * phase_fix_observable) @ state
#                     else:
#                         state *= 0
# 
#                 if m == 0 and k == 0:
#                     exp_state = state
#                 else:
#                     exp_state += state
#         if not phase_fix_states:
#             exp_states = [exp_state]
#         else:
#             exp_states += [exp_state]
# 
#     # Return the composed states
#     return ListOp(exp_states)
# 
# 
#     dH_qfi = []
#     for coeff_i in H_coeffs:
#         dH_qfi_term = []
#         for param_s in params:
#             for operators in qfi.oplist:
#                 for operator in operators:
#                     if isinstance(operator, ComposedOp):
#                         meas = operator[0]
#                         state = operator[1]
#                         exp1 = meas @ get_state_exp(state, param_s)
#                         exp2 = meas @ get_state_exp(state, param_s, exp=2)
#                         dH_qfi_term += [dH_params * (exp1 + exp2)]
#         dH_qfi.append(SummedOp(dH_qfi_term))
#     return ListOp(dH_qfi)
